# Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict

import warnings
warnings.filterwarnings('ignore')

# Load the Train Test Split

In [9]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12) 
Testing Samples : (83, 250, 12)
Training Labels : (332,) 
Testing Labels : (83,)


In [10]:
_ , num_timeframes, num_features = X_train.shape

# Reshape data for the model
-----------------
Samples are a 2D matrix which need to be reshaped to 1D to feed into the model

In [11]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 3000) 
Testing Samples : (83, 3000)
Training Labels : (332,) 
Testing Labels : (83,)


# Grid search over SVM_OVO model

In [12]:
parameters = {
    "kernel": ['rbf'],
    "C": np.logspace(2, 4, 10), 
    "gamma": np.logspace(-4, -2, 10)
}

grid = GridSearchCV(SVC(decision_function_shape='ovo'), parameters, cv=3, verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] C=100.0, gamma=0.0001, kernel=rbf ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ C=100.0, gamma=0.0001, kernel=rbf, total=   0.2s
[CV] C=100.0, gamma=0.0001, kernel=rbf ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ................ C=100.0, gamma=0.0001, kernel=rbf, total=   0.2s
[CV] C=100.0, gamma=0.0001, kernel=rbf ...............................
[CV] ................ C=100.0, gamma=0.0001, kernel=rbf, total=   0.2s
[CV] C=100.0, gamma=0.0001668100537200059, kernel=rbf ................
[CV] . C=100.0, gamma=0.0001668100537200059, kernel=rbf, total=   0.2s
[CV] C=100.0, gamma=0.0001668100537200059, kernel=rbf ................
[CV] . C=100.0, gamma=0.0001668100537200059, kernel=rbf, total=   0.2s
[CV] C=100.0, gamma=0.0001668100537200059, kernel=rbf ................
[CV] . C=100.0, gamma=0.0001668100537200059, kernel=rbf, total=   0.2s
[CV] C=100.0, gamma=0.0002782559402207126, kernel=rbf ................
[CV] . C=100.0, gamma=0.0002782559402207126, kernel=rbf, total=   0.2s
[CV] C=100.0, gamma=0.0002782559402207126, kernel=rbf ................
[CV] . C=100.0, gamma=0.0002782559402207126, kernel=rbf, total=   0.2s
[CV] C=100.0, gamma=0.0002782559402207126, kernel=rbf ................
[CV] .

...

[CV]  C=1291.549665014884, gamma=0.001291549665014884, kernel=rbf, total=   0.4s
[CV] C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf, total=   0.5s
[CV] C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf, total=   0.4s
[CV] C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf, total=   0.5s
[CV] C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf, total=   0.5s
[CV] C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf, total=   0.5s
[CV] C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf, total=   0.5s
[CV] C=

[CV]  C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf, total=   0.4s
[CV] C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf, total=   0.4s
[CV] C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf, total=   0.4s
[CV] C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf, total=   0.4s
[CV] C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf, total=   0.4s
[CV] C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf, total=   0.5s
[CV] C=3593.813663804626, gamma=0.002154434690031882, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.002154434690031882, kernel=rbf, total=   0.5s
[CV] C=

[CV]  C=10000.0, gamma=0.0002782559402207126, kernel=rbf, total=   0.4s
[CV] C=10000.0, gamma=0.0002782559402207126, kernel=rbf ..............
[CV]  C=10000.0, gamma=0.0002782559402207126, kernel=rbf, total=   0.4s
[CV] C=10000.0, gamma=0.00046415888336127773, kernel=rbf .............
[CV]  C=10000.0, gamma=0.00046415888336127773, kernel=rbf, total=   0.4s
[CV] C=10000.0, gamma=0.00046415888336127773, kernel=rbf .............
[CV]  C=10000.0, gamma=0.00046415888336127773, kernel=rbf, total=   0.4s
[CV] C=10000.0, gamma=0.00046415888336127773, kernel=rbf .............
[CV]  C=10000.0, gamma=0.00046415888336127773, kernel=rbf, total=   0.4s
[CV] C=10000.0, gamma=0.000774263682681127, kernel=rbf ...............
[CV]  C=10000.0, gamma=0.000774263682681127, kernel=rbf, total=   0.4s
[CV] C=10000.0, gamma=0.000774263682681127, kernel=rbf ...............
[CV]  C=10000.0, gamma=0.000774263682681127, kernel=rbf, total=   0.4s
[CV] C=10000.0, gamma=0.000774263682681127, kernel=rbf ..............

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.0min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovo', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([  100.        ,   166.81005372,   278.25594022,   464.15888336,
         774.26368268,  1291.54966501,  2154.43469003,  3593.8136638 ,
        5994.84250319, 10000.        ]),
                         'gamma': array([0.0001    , 0.00016681, 0.00027826, 0.00046416, 0.00077426,
       0.00129155, 0.00215443, 0.00359381, 0.00599484, 0.01      ]),
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scorin

In [13]:
params = grid.best_params_
best_estimator = grid.best_estimator_
print(params)
print(best_estimator)

{'C': 100.0, 'gamma': 0.002154434690031882, 'kernel': 'rbf'}
SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.002154434690031882,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


In [14]:
clf = best_estimator
clf.fit(X_train,Y_train)

SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.002154434690031882,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [15]:
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.80      0.92      0.86        13
           1       0.86      0.80      0.83        15
           2       0.92      0.79      0.85        14
           3       1.00      1.00      1.00         9
           4       0.70      0.80      0.74        20
           5       0.60      0.50      0.55        12

    accuracy                           0.80        83
   macro avg       0.81      0.80      0.80        83
weighted avg       0.80      0.80      0.79        83

[[12  1  0  0  0  0]
 [ 3 12  0  0  0  0]
 [ 0  0 11  0  2  1]
 [ 0  0  0  9  0  0]
 [ 0  0  1  0 16  3]
 [ 0  1  0  0  5  6]]


# Train on entire data

In [28]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))

X_train = np.concatenate((X_train,X_test), axis=0)
Y_train = np.concatenate((Y_train,Y_test), axis=0)

_ , num_timeframes, num_features = X_train.shape
X_train = X_train.reshape((-1,num_timeframes*num_features))
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(X_train)

from sklearn.utils import shuffle
X_train, X_sparse, Y_train = shuffle(X_train, X_sparse, Y_train)
X_train = X_train.reshape((-1,num_timeframes,num_features))

In [29]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (415, 3000) 
Testing Samples : (83, 250, 12)
Training Labels : (415,) 
Testing Labels : (83,)


In [30]:
parameters = {
    "kernel": ['rbf'],
    "C": np.logspace(2, 4, 10), 
    "gamma": np.logspace(-4, -2, 10)
}

grid = GridSearchCV(SVC(decision_function_shape='ovo'), parameters, cv=3, verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] C=100.0, gamma=0.0001, kernel=rbf ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ C=100.0, gamma=0.0001, kernel=rbf, total=   0.4s
[CV] C=100.0, gamma=0.0001, kernel=rbf ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ................ C=100.0, gamma=0.0001, kernel=rbf, total=   0.4s
[CV] C=100.0, gamma=0.0001, kernel=rbf ...............................
[CV] ................ C=100.0, gamma=0.0001, kernel=rbf, total=   0.4s
[CV] C=100.0, gamma=0.0001668100537200059, kernel=rbf ................
[CV] . C=100.0, gamma=0.0001668100537200059, kernel=rbf, total=   0.4s
[CV] C=100.0, gamma=0.0001668100537200059, kernel=rbf ................
[CV] . C=100.0, gamma=0.0001668100537200059, kernel=rbf, total=   0.4s
[CV] C=100.0, gamma=0.0001668100537200059, kernel=rbf ................
[CV] . C=100.0, gamma=0.0001668100537200059, kernel=rbf, total=   0.4s
[CV] C=100.0, gamma=0.0002782559402207126, kernel=rbf ................
[CV] . C=100.0, gamma=0.0002782559402207126, kernel=rbf, total=   0.4s
[CV] C=100.0, gamma=0.0002782559402207126, kernel=rbf ................
[CV] . C=100.0, gamma=0.0002782559402207126, kernel=rbf, total=   0.4s
[CV] C=100.0, gamma=0.0002782559402207126, kernel=rbf ................
[CV] .

...

[CV]  C=1291.549665014884, gamma=0.001291549665014884, kernel=rbf, total=   0.3s
[CV] C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf, total=   0.4s
[CV] C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf, total=   0.3s
[CV] C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.002154434690031882, kernel=rbf, total=   0.3s
[CV] C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf, total=   0.4s
[CV] C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf, total=   0.4s
[CV] C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf .....
[CV]  C=1291.549665014884, gamma=0.003593813663804626, kernel=rbf, total=   0.4s
[CV] C=

[CV]  C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf, total=   0.3s
[CV] C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf, total=   0.3s
[CV] C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.000774263682681127, kernel=rbf, total=   0.3s
[CV] C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf, total=   0.3s
[CV] C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf, total=   0.3s
[CV] C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.001291549665014884, kernel=rbf, total=   0.3s
[CV] C=3593.813663804626, gamma=0.002154434690031882, kernel=rbf .....
[CV]  C=3593.813663804626, gamma=0.002154434690031882, kernel=rbf, total=   0.4s
[CV] C=

[CV]  C=10000.0, gamma=0.0002782559402207126, kernel=rbf, total=   0.3s
[CV] C=10000.0, gamma=0.0002782559402207126, kernel=rbf ..............
[CV]  C=10000.0, gamma=0.0002782559402207126, kernel=rbf, total=   0.3s
[CV] C=10000.0, gamma=0.00046415888336127773, kernel=rbf .............
[CV]  C=10000.0, gamma=0.00046415888336127773, kernel=rbf, total=   0.3s
[CV] C=10000.0, gamma=0.00046415888336127773, kernel=rbf .............
[CV]  C=10000.0, gamma=0.00046415888336127773, kernel=rbf, total=   0.3s
[CV] C=10000.0, gamma=0.00046415888336127773, kernel=rbf .............
[CV]  C=10000.0, gamma=0.00046415888336127773, kernel=rbf, total=   0.3s
[CV] C=10000.0, gamma=0.000774263682681127, kernel=rbf ...............
[CV]  C=10000.0, gamma=0.000774263682681127, kernel=rbf, total=   0.3s
[CV] C=10000.0, gamma=0.000774263682681127, kernel=rbf ...............
[CV]  C=10000.0, gamma=0.000774263682681127, kernel=rbf, total=   0.3s
[CV] C=10000.0, gamma=0.000774263682681127, kernel=rbf ..............

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  1.9min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovo', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([  100.        ,   166.81005372,   278.25594022,   464.15888336,
         774.26368268,  1291.54966501,  2154.43469003,  3593.8136638 ,
        5994.84250319, 10000.        ]),
                         'gamma': array([0.0001    , 0.00016681, 0.00027826, 0.00046416, 0.00077426,
       0.00129155, 0.00215443, 0.00359381, 0.00599484, 0.01      ]),
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scorin

In [31]:
clf = grid.best_estimator_
clf.fit(X_train,Y_train)

SVC(C=166.81005372000593, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovo', degree=3,
    gamma=0.002154434690031882, kernel='rbf', max_iter=-1, probability=False,
    random_state=None, shrinking=True, tol=0.001, verbose=False)

# Saving model

In [32]:
pickle.dump(clf, open(os.path.join("..", "IPD", "SVM-OVO_model.pkl"), 'wb'))
print("Model Saved")

Model Saved
